# Deep Learning Project

## Milestone report

## Study of the impact on the ratio of labeled to unlabeled data on top-1 accuracy on the MNIST dataset

Pierre Andurand (pa2570)
Tzu Yi Chuang (tc3075)
Kuan Yu Ko (kk3376)


In [9]:
#This part is not needed for now

# Import standard libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

# Import TF layers
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D

# Import TF utilities
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Import TF pretrained models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.nasnet import NASNetMobile

Below we train a simple model in supervised learning without data augmentation. We check its performance on the mnist dataset. It will be our un-noised teacher model. And we will compare its performance with 100 epochs to the semi supervised self-learning model in the following block

In [52]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np

batch_size = 200
num_classes = 10
epochs = 100
data_augmentation = False
#num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
teacher_name = 'keras_mnist_trained_teacher.h5'

# Load the data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train=x_train.reshape(x_train.shape[0],28,28,1).astype('float32')/255
x_test=x_test.reshape(x_test.shape[0],28,28,1).astype('float32')/255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

teacher = Sequential()
teacher.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
teacher.add(Activation('relu'))
teacher.add(Conv2D(32, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Conv2D(64, (3, 3), padding='same'))
teacher.add(Activation('relu'))
teacher.add(Conv2D(64, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Flatten())
teacher.add(Dense(512))
teacher.add(Activation('relu'))
#model.add(Dropout(0.5))
teacher.add(Dense(num_classes))
teacher.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compiling the model using RMSprop
teacher.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#Training the model

if not data_augmentation:
    print('Not using data augmentation.')
    teacher.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    teacher.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_split=0.2,
                        workers=4)
    
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
teacher_path = os.path.join(save_dir, teacher_name)
teacher.save(teacher_path)
print('Saved trained model at %s ' % teacher_path)

# Score trained model.
scores = teacher.evaluate(x_test, y_test, verbose=1)
print('Supervised learning model with '+str(epochs)+'epochs \n')
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Not using data augmentation.
Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 60s 1ms/step - loss: 2.0303 - accuracy: 0.4059 - val_loss: 1.2557 - val_accuracy: 0.7395
Epoch 2/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.8239 - accuracy: 0.7637 - val_loss: 0.5211 - val_accuracy: 0.8427
Epoch 3/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.5391 - accuracy: 0.8336 - val_loss: 0.3956 - val_accuracy: 0.8853
Epoch 4/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.4334 - accuracy: 0.8669 - val_loss: 0.3262 - val_accuracy: 0.8991
Epoch 5/100
48000/48000 [==============================] - 59s 1ms/step - loss: 0.3500 - accuracy: 0.8931 - val_loss: 0.2621 - val_accuracy: 0.9237
Epoch 6/100
48000/48000 [==============================] - 59s 1ms/step - loss: 0.2837 - accuracy: 0.9118 - 

48000/48000 [==============================] - 60s 1ms/step - loss: 0.0181 - accuracy: 0.9942 - val_loss: 0.0387 - val_accuracy: 0.9899
Epoch 56/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0183 - accuracy: 0.9940 - val_loss: 0.0378 - val_accuracy: 0.9903
Epoch 57/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0171 - accuracy: 0.9945 - val_loss: 0.0410 - val_accuracy: 0.9898
Epoch 58/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0161 - accuracy: 0.9944 - val_loss: 0.0382 - val_accuracy: 0.9898
Epoch 59/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0161 - accuracy: 0.9944 - val_loss: 0.0390 - val_accuracy: 0.9896
Epoch 60/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0158 - accuracy: 0.9945 - val_loss: 0.0390 - val_accuracy: 0.9898
Epoch 61/100
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0158 - accuracy: 0.9947 - val_loss: 

Below we study the semi supervised learning method as described in the article, and study the impact of the ratio of true labels to pseudo labels on accuracy and loss. 
We do a loop over different ratios of label to unlabelled data (rate). And each loop does the following:
1) Train un-noised model (teacher) on labeled data only 
2) Ten cycles of: un-noised model (teacher)->predict hard pseudolabel->training 10 epochs for noised model (student=teacher+dropout noise) on labeled+pseudo labeled->new weights. 

We check rates from 0.1 to 50. The student model will be the teacher model noised by a Dropout(0.5) before the last layer

In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os

#batch_size = 32
num_classes = 10
#epochs = 50
#data_augmentation = False
#num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
teacher_name = 'keras_mnist_trained_teacher.h5'

teacher_path = os.path.join(save_dir, teacher_name)

# Load the mnist data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train=x_train.reshape(x_train.shape[0],28,28,1).astype('float32')/255
x_test=x_test.reshape(x_test.shape[0],28,28,1).astype('float32')/255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


#making un-noised teacher model
teacher = Sequential()
teacher.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
teacher.add(Activation('relu'))
teacher.add(Conv2D(32, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Conv2D(64, (3, 3), padding='same'))
teacher.add(Activation('relu'))
teacher.add(Conv2D(64, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Flatten())
teacher.add(Dense(512))
teacher.add(Activation('relu'))
#model.add(Dropout(0.5)) #this will be uncommented for the noised student model
teacher.add(Dense(num_classes))
teacher.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compile the teacher model using RMSprop
teacher.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])





#noised student model

student = Sequential()
student.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
student.add(Activation('relu'))
student.add(Conv2D(32, (3, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))
student.add(Dropout(0.25))

student.add(Conv2D(64, (3, 3), padding='same'))
student.add(Activation('relu'))
student.add(Conv2D(64, (3, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))
student.add(Dropout(0.25))

student.add(Flatten())
student.add(Dense(512))
student.add(Activation('relu'))
student.add(Dropout(0.5))
student.add(Dense(num_classes))
student.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compiling the model using RMSprop
student.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])





# rate= n_true/n_pseudo
rate=np.array([0.1,0.2,0.3,0.5,0.75,1,2,3,4,5,10,20,30,50])
n_total=x_train.shape[0]

# total number of train images (n_total) = number of true label images (n_true) + number of pseudo label images (n_pseudo)
#                                        = n_pseudo(rate+1)
# n_pseudo = n_total/(rate+1); n_true=n_total-n_pseudo

#loop over rate values in order to find the optimal rate value for the self-learning semi supervised learning, 
#ie one that will maximize accuracy
for r in rate:
    print("rate="+str(r)+":\n")
    n_pseudo=n_total/(1+r)
    n_true=n_total-n_pseudo
    print(n_pseudo)
    print(n_true)
    mask_true=np.random.choice(int(n_total),int(n_true),replace=False) #generating n_true integers between 0 and n_total-1
    mask_pseudo=[item for item in range(n_total) if item not in mask_true] #all the other numbers between 0 and n_total-1 which are not in mask_true
    mask_pseudo=np.array(mask_pseudo)
    x_true=x_train[mask_true[:]] #x for the labeled data
    y_true=y_train[mask_true[:]] #y for the labeled data
    x_pseudo=x_train[mask_pseudo[:]] #x for the unlabeled data (pseudo)
    #training teacher model on labeled data with validation split of 0.2
    training=teacher.fit(x_true,y_true,validation_split=0.2,
                            epochs=10,batch_size=200,verbose=0)
    #evaluating teacher model on test data    
    scores=teacher.evaluate(x_test,y_test,verbose=0)
    print("Original model with labelled data only predicting on test data: ",scores[1])

    x_true_pseudo=np.concatenate([x_true,x_pseudo]) #concatenating x for labeled and unlabeled data
    print('x_true_pseudo.shape: ',x_true_pseudo.shape)
    prediction=teacher.predict_classes(x_pseudo) #predicting labels on unlabeled data
    y_pseudo=keras.utils.to_categorical(prediction, num_classes)
    y_true_pseudo=np.concatenate([y_true,y_pseudo]) #concatenating y for labeled and pseudo labeled
    print('y_true_pseudo.shape: ', y_true_pseudo.shape)
    for i in range(10): 
        # 10 loops of 10 epochs of noised student training for labeled and pseudo labeled data (step 3 in article)
        # followed by generating predictions on unlabeled data with the teacher model (=un-noised student)
        # which uses the weights of the trained noised student (noise does not change the weights structure of models) (step 2 in article)
        print(i)
        training=student.fit(x_true_pseudo,y_true_pseudo,validation_split=0.2,
                             epochs=10,batch_size=200,verbose=0)
        # Save weights
        student.save_weights(teacher_path)
        # Load weights for teacher model (un-noised)
        teacher.load_weights(teacher_path)
        prediction=teacher.predict_classes(x_pseudo)
        scores=teacher.evaluate(x_test,y_test,verbose=0) #evaluating model on test data
        print('iteration: ',i)
        print('Test loss:', scores[0])
        print('Test accuracy:', scores[1])
        y_pseudo=keras.utils.to_categorical(prediction, num_classes)
        y_true_pseudo=np.concatenate([y_true,y_pseudo]) #new y_true_pseudo to be used in next loop
        




x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
rate=0.1:

54545.454545454544
5454.545454545456
Original model with labelled data only predicting on test data:  0.9182000160217285
x_true_pseudo.shape:  (60000, 28, 28, 1)
y_true_pseudo.shape:  (60000, 10)
0
iteration:  0
Test loss: 0.22905886516885365
Test accuracy: 0.9355999827384949
1
iteration:  1
Test loss: 0.20284845472330054
Test accuracy: 0.9463000297546387
2
iteration:  2
Test loss: 0.1859398955702636
Test accuracy: 0.9534000158309937
3
iteration:  3
Test loss: 0.17726550295493654
Test accuracy: 0.9575999975204468
4
iteration:  4
Test loss: 0.17548563856298752
Test accuracy: 0.9596999883651733
5
iteration:  5
Test loss: 0.16725050273136366
Test accuracy: 0.9605000019073486
6
iteration:  6
Test loss: 0.16760790004320333
Test accuracy: 0.9635999798774719
7
iteration:  7
Test loss: 0.17377933695675146
Test accuracy: 0.963699996471405
8
iteration:  8
Test loss: 0.17724166210107947
Test accuracy: 0.9645000100

y_true_pseudo.shape:  (60000, 10)
0
iteration:  0
Test loss: 0.02283670855613891
Test accuracy: 0.9937999844551086
1
iteration:  1
Test loss: 0.016973818930707058
Test accuracy: 0.9947999715805054
2
iteration:  2
Test loss: 0.0190154507946223
Test accuracy: 0.9944999814033508
3
iteration:  3
Test loss: 0.018861525846802396
Test accuracy: 0.9945999979972839
4
iteration:  4
Test loss: 0.019169638458127157
Test accuracy: 0.9937999844551086
5
iteration:  5
Test loss: 0.017974098205029442
Test accuracy: 0.9944999814033508
6
iteration:  6
Test loss: 0.023868342255335302
Test accuracy: 0.9947999715805054
7
iteration:  7
Test loss: 0.020866385617130435
Test accuracy: 0.9943000078201294
8
iteration:  8
Test loss: 0.023016728306739242
Test accuracy: 0.9930999875068665
9
iteration:  9
Test loss: 0.016992215464339096
Test accuracy: 0.9947999715805054
rate=5.0:

10000.0
50000.0
Original model with labelled data only predicting on test data:  0.9930999875068665
x_true_pseudo.shape:  (60000, 28, 28, 

KeyboardInterrupt: 

We can conclude that a ratio of labeled/unlabeled above 0.3 adds some accuracy on the full training set of MNIST. It means that having less than one third of the total training set kept as unlabeled data helps the accuracy relative to the fully supervised version. Below we will do the same experiment as above, but taking a small sample of the MNIST dataset, with only 100 images in total from the training dataset, and its accuracy calculated against the full test dataset. 

In [62]:
#supervised learning on small training dataset, testing on full testing dataset


from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np

#batch_size = 32
num_classes = 10
#epochs = 50
#data_augmentation = False
#num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
teacher_name = 'keras_mnist_trained_teacher.h5'

teacher_path = os.path.join(save_dir, teacher_name)

#making un-noised teacher model
teacher = Sequential()
teacher.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
teacher.add(Activation('relu'))
teacher.add(Conv2D(32, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Conv2D(64, (3, 3), padding='same'))
teacher.add(Activation('relu'))
teacher.add(Conv2D(64, (3, 3)))
teacher.add(Activation('relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))
teacher.add(Dropout(0.25))

teacher.add(Flatten())
teacher.add(Dense(512))
teacher.add(Activation('relu'))
#model.add(Dropout(0.5)) #this will be uncommented for the noised student model
teacher.add(Dense(num_classes))
teacher.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compile the teacher model using RMSprop
teacher.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

#noised student model

student = Sequential()
student.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
student.add(Activation('relu'))
student.add(Conv2D(32, (3, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))
student.add(Dropout(0.25))

student.add(Conv2D(64, (3, 3), padding='same'))
student.add(Activation('relu'))
student.add(Conv2D(64, (3, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))
student.add(Dropout(0.25))

student.add(Flatten())
student.add(Dense(512))
student.add(Activation('relu'))
student.add(Dropout(0.5))
student.add(Dense(num_classes))
student.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Compiling the model using RMSprop
student.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



# Load the mnist data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train=x_train.reshape(x_train.shape[0],28,28,1).astype('float32')/255
x_test=x_test.reshape(x_test.shape[0],28,28,1).astype('float32')/255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(y_train.shape)

#checking that classes are balanced

#print(np.unique(y_train))
#print(y_train[0:40])

sample_size=100
for i in range(10):
    print(str(i)+":",sum(y_train==i))

#Selecting 10 images of each class
k=0
x_small_train=np.zeros((sample_size,28,28,1))
y_small_train=np.full((sample_size,),-1)

for i in range(x_train.shape[0]):
    #print(i)
    for j in range(10):
        if sum(y_small_train==j)<sample_size/10:
            if y_train[i]==j:
                x_small_train[k,:]=x_train[i,:]
                y_small_train[k]=y_train[i]
                k+=1
                break
    #print('k=',k)
    if k==sample_size:
        break
        
#print(y_small_train[0:40])
print(x_small_train.shape)
print(y_small_train.shape)

#verifying that there are 10 images in each class
for i in range(10):
    print(str(i)+":",sum(y_small_train==i))
    
# Convert class vectors to binary class matrices.
y_small_train = keras.utils.to_categorical(y_small_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)    

#train teacher model
teacher.fit(x_small_train, y_small_train,
            batch_size=10,
            epochs=10,
            validation_split=0.,
            shuffle=True)


# Score trained model.
scores = teacher.evaluate(x_test, y_test, verbose=1)
print('Small sample of 100 training images, Supervised learning model with '+str(10)+'epochs \n')
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(60000,)
0: 5923
1: 6742
2: 5958
3: 6131
4: 5842
5: 5421
6: 5918
7: 6265
8: 5851
9: 5949
(100, 28, 28, 1)
(100,)
0: 10
1: 10
2: 10
3: 10
4: 10
5: 10
6: 10
7: 10
8: 10
9: 10
Epoch 1/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
100/100 [======

The accuracy result above is surprisingly good for such a small dataset. Now we keep the same labeled training set of 100 images, and add an unlabeled training set determined by the ratio and run STSN

In [56]:
# Varying the ratio for 100 labeled images. Rest of training dataset unlabeled with ratio determining size of total dataset.
# Testing on full test dataset

# Load the mnist data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train=x_train.reshape(x_train.shape[0],28,28,1).astype('float32')/255
x_test=x_test.reshape(x_test.shape[0],28,28,1).astype('float32')/255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(y_train.shape)

#checking that classes are balanced

#print(np.unique(y_train))
#print(y_train[0:40])

sample_size=100
for i in range(10):
    print(str(i)+":",sum(y_train==i))

#Selecting 10 images of each class
k=0
x_small_train=np.zeros((sample_size,28,28,1))
y_small_train=np.full((sample_size,),-1)

for i in range(x_train.shape[0]):
    #print(i)
    for j in range(10):
        if sum(y_small_train==j)<sample_size/10:
            if y_train[i]==j:
                x_small_train[k,:]=x_train[i,:]
                y_small_train[k]=y_train[i]
                k+=1
                break
    #print('k=',k)
    if k==sample_size:
        break
        
#print(y_small_train[0:40])
print(x_small_train.shape)
print(y_small_train.shape)

#verifying that there are 10 images in each class
for i in range(10):
    print(str(i)+":",sum(y_small_train==i))
    
# Convert class vectors to binary class matrices.
y_small_train = keras.utils.to_categorical(y_small_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)    





# rate= n_true/n_pseudo, n_true=sample_size
rate=np.array([0.1,0.2,0.3,0.5,0.75,1,2,3,4,5,10,20,30,50])
#n_total=sample_size*(1+1/rate)

# total number of train images (n_total) = number of true label images (sample_size) + number of pseudo label images (n_pseudo)
#                                        = sample_size(1/rate+1)
# n_pseudo = sample_size/rate

#loop over rate values in order to find the optimal rate value for the self-learning semi supervised learning, 
#ie one that will maximize accuracy
for r in rate:
    print("rate="+str(r)+":\n")
    n_pseudo=int(sample_size/r)
    n_true=sample_size
    n_total=n_true+n_pseudo
    print(n_pseudo, n_true, n_total)
    x_true=x_small_train
    y_true=y_small_train
    x_pseudo=x_train[500:500+n_pseudo,:] #x for the unlabeled data (pseudo)
    #training teacher model on labeled data with validation split of 0.2
    training=teacher.fit(x_true,y_true,validation_split=0.,
                            epochs=10,batch_size=int(n_total/10),verbose=0)
    #evaluating teacher model on test data    
    scores=teacher.evaluate(x_test,y_test,verbose=0)
    print("Original model with labelled data only predicting on test data: ",scores[1])

    x_true_pseudo=np.concatenate([x_true,x_pseudo]) #concatenating x for labeled and unlabeled data
    print('x_true_pseudo.shape: ',x_true_pseudo.shape)
    prediction=teacher.predict_classes(x_pseudo) #predicting labels on unlabeled data
    y_pseudo=keras.utils.to_categorical(prediction, num_classes)
    y_true_pseudo=np.concatenate([y_true,y_pseudo]) #concatenating y for labeled and pseudo labeled
    print('y_true_pseudo.shape: ', y_true_pseudo.shape)
    for i in range(10): 
        # 10 loops of 10 epochs of noised student training for labeled and pseudo labeled data (step 3 in article)
        # followed by generating predictions on unlabeled data with the teacher model (=un-noised student)
        # which uses the weights of the trained noised student (noise does not change the weights structure of models) (step 2 in article)
        print(i)
        training=student.fit(x_true_pseudo,y_true_pseudo,validation_split=0.,
                             epochs=10,batch_size=int(n_total/10),verbose=0)
        # Save weights
        student.save_weights(teacher_path)
        # Load weights for teacher model (un-noised)
        teacher.load_weights(teacher_path)
        prediction=teacher.predict_classes(x_pseudo)
        scores=teacher.evaluate(x_test,y_test,verbose=0) #evaluating model on test data
        print('iteration: ',i)
        print('Test loss:', scores[0])
        print('Test accuracy:', scores[1])
        y_pseudo=keras.utils.to_categorical(prediction, num_classes)
        y_true_pseudo=np.concatenate([y_true,y_pseudo]) #new y_true_pseudo to be used in next loop
        


#from keras.utils import plot_model
#plot_model(teacher,to_file='teacher.png')
#plot_model(student,to_file='student.png')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(60000,)
0: 5923
1: 6742
2: 5958
3: 6131
4: 5842
5: 5421
6: 5918
7: 6265
8: 5851
9: 5949
(100, 28, 28, 1)
(100,)
0: 10
1: 10
2: 10
3: 10
4: 10
5: 10
6: 10
7: 10
8: 10
9: 10
rate=0.1:

1000 100 1100
Original model with labelled data only predicting on test data:  0.9911999702453613
x_true_pseudo.shape:  (1100, 28, 28, 1)
y_true_pseudo.shape:  (1100, 10)
0
iteration:  0
Test loss: 0.024838309279835994
Test accuracy: 0.9932000041007996
1
iteration:  1
Test loss: 0.024450271297374183
Test accuracy: 0.9927999973297119
2
iteration:  2
Test loss: 0.029390712049562718
Test accuracy: 0.9926000237464905
3
iteration:  3
Test loss: 0.028170187339861333
Test accuracy: 0.9926000237464905
4
iteration:  4
Test loss: 0.023448410414327304
Test accuracy: 0.9936000108718872
5
iteration:  5
Test loss: 0.024384515968289885
Test accuracy: 0.9941999912261963
6
iteration:  6
Test loss: 0.025380505136862866
Test accuracy: 0.993399977684021

iteration:  0
Test loss: 0.0680462593742767
Test accuracy: 0.9901000261306763
1
iteration:  1
Test loss: 0.06672358556740099
Test accuracy: 0.9900000095367432
2
iteration:  2
Test loss: 0.1134485397388905
Test accuracy: 0.9897000193595886
3
iteration:  3
Test loss: 0.14285790954044048
Test accuracy: 0.9848999977111816
4
iteration:  4
Test loss: 0.12335549907723048
Test accuracy: 0.9876999855041504
5
iteration:  5
Test loss: 0.1313646703713602
Test accuracy: 0.9865000247955322
6
iteration:  6
Test loss: 0.13406273119041112
Test accuracy: 0.9866999983787537
7
iteration:  7
Test loss: 0.16575898618696852
Test accuracy: 0.9857000112533569
8
iteration:  8
Test loss: 0.14278378531077518
Test accuracy: 0.9886999726295471
9
iteration:  9
Test loss: 0.07838338035165202
Test accuracy: 0.9889000058174133
rate=5.0:

20 100 120
Original model with labelled data only predicting on test data:  0.9901000261306763
x_true_pseudo.shape:  (120, 28, 28, 1)
y_true_pseudo.shape:  (120, 10)
0
iteration:  0
Te

The ratio from 0.1 to 0.75 helps the accuracy marginally. It is just difficult to do extensive studies on MNIST as the accuracy with a simple model is already very high

In [20]:
pip install pydot

In [5]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.
